In [10]:
# https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

import pandas as pd
import nltk
import json

# download nltk corpus (first time only)
# nltk.download("all")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
# Installiere das SnowballStemmer für die deutsche Sprache
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
analyzer = SentimentIntensityAnalyzer()

## Helper functions

In [72]:
# the columns are beeing merged togheter here
def merge_columns_in_dataframe(df):
    columns_to_merge = ['page_title', 'sub_title', 'introduction', 'summary_box', 'content', 'accordion']
    new_df = df.loc[:, ["url", "page_title"]].copy()
    new_df['text'] = df.apply(lambda row: ' '.join([str(row[col]) for col in columns_to_merge]), axis=1)
    return new_df

In [128]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
    stemmer = SnowballStemmer("german") # eventuell anderen stemmer verwenden
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    processed_text = " ".join(stemmed_tokens)
    return processed_text

In [125]:
def get_sentiment(text_string):
    scores = analyzer.polarity_scores(text_string)
    return pd.Series(scores)

def add_sentiment_cols(dataframe, text_col="text"):
    sentiment_cols = dataframe[text_col].apply(get_sentiment)
    dataframe = pd.concat([dataframe, sentiment_cols], axis=1)
    dataframe['judgement'] = dataframe['compound'].apply(lambda x: 'positive' if x > 0.05 else ("negative" if x < -0.05 else 'neutral'))
    return dataframe

In [126]:
def get_analysis_data(json_file):
    pandas_df = pd.read_json(json_file)
    filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'page_title'])] != "").any(axis=1)] # exkludiert alle spalten von der Analyse die "" in den spalten ausser "page_title" haben 
    data = merge_columns_in_dataframe(filtered_df)
    data["text"] = data["text"].apply(preprocess_text)
    return add_sentiment_cols(data)

In [9]:
def save_as_json(pandas_dataframe, file_name):
    import datetime
    date_info = datetime.datetime.now().strftime("%d-%m-%y")
    pandas_dataframe.to_json(f"{file_name}_{date_info}.json", orient='records')

In [10]:
# df = get_analysis_data("ratgeber_pages_part2.json")
# save_as_json(df, "ratgeber_pages_with_rating")

## Execution of Code

In [ ]:
# df2 = get_analysis_data("../web-crawler/scrapy_mobiliar/mobiscraper/mobiscraper/spiders/scrape_archive/full_scrape_2.json")
# save_as_json(df2, "full_scrape_2_analysis")

In [12]:
df2 = pd.read_json("full_scrape_2_analysis_15-03-24_15:40:08.json")
display(df2)

,url,page_title,text,neg,neu,pos,compound,judgement
0,https://www.mobiliar.ch/versicherungen-und-vor...,Tierversicherung,tierversicher hund katz versich tierversicher ...,0.000,0.994,0.006,0.2406,positive
1,https://www.mobiliar.ch/versicherungen-und-vor...,Cyberversicherung,cyberversicher hilf digital alltag bankgeschaf...,0.028,0.954,0.019,-0.5362,negative
2,https://www.mobiliar.ch/versicherungen-und-vor...,Wertsachenversicherung,"wertsachenversicher versich , bedeutet wertvol...",0.000,0.981,0.019,0.6553,positive
3,https://www.mobiliar.ch/versicherungen-und-vor...,Die Hausratversicherung,hausratversicher rundum-versicher beweg sach w...,0.021,0.979,0.000,-0.6662,negative
4,https://www.mobiliar.ch/versicherungen-und-vor...,Mietkautionsversicherung,mietkautionsversicher dank mietkaution spar mi...,0.007,0.976,0.017,0.4329,positive
...,...,...,...,...,...,...,...,...
1456,https://www.mobiliar.ch/versicherungen-und-vor...,Die Fahrrad-Kaskoversicherung vermitteln,fahrrad-kaskoversicher vermitteln umfass versi...,0.028,0.972,0.000,-0.3313,negative
1457,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Projekte 2022: Insgesamt 19 weitere Brücken un...,projekt 2022 : insgesamt 19 weit bruck steg fe...,0.000,1.000,0.000,0.0000,neutral
1458,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Projekte 2023: Insgesamt 13 weitere Brücken un...,projekt 2023 : insgesamt 13 weit bruck steg fe...,0.000,1.000,0.000,0.0000,neutral
1459,https://www.mobiliar.ch/versicherungen-und-vor...,Winter Wanderland,"wint wanderland verschneit tal , weit nebelme ...",0.000,1.000,0.000,0.0000,neutral


In [13]:
df2["judgement"].value_counts()

judgement
neutral     726
positive    465
negative    270
Name: count, dtype: int64

## to display the Functions

In [14]:
json_file_example = [{"url": "https://www.mobiliar.ch/die-mobiliar/ueber-uns/unsere-werbung/werbespots-liebe-mobiliar/liebe-mobiliar-parkett", "page_title": "Liebe Mobiliar: Parkett", "sub_title": "", "introduction": "Der neue «Liebe Mobiliar»-Spot zeigt, dass Schäden zu Hause immer genau dann passieren, wenn man nicht damit rechnet.", "summary_box": "", "content": "Die Mobiliar startet mit dem neusten «Liebe Mobiliar»-Spot ins neue Jahr. Die Familie im Spot hingegen startet in einen nur auf den ersten Blick gewöhnlichen Sonntagmorgen. Denn: Während die Eltern noch gemütlich vor sich hinschlummern, werden sie von Geräuschen geweckt, die definitiv nicht ins Haus gehören. Grund dafür ist ihre Tochter, die ihr Talent als Eiskunstläuferin entdeckt und mit viel Freude auslebt. Und das hinterlässt Spuren. Was immer kommt – wir helfen Ihnen rasch und unkompliziert.", "accordion": ""},]

In [15]:
pandas_dataframe = pd.DataFrame(json_file_example)
pandas_dataframe

,url,page_title,sub_title,introduction,summary_box,content,accordion
0,https://www.mobiliar.ch/die-mobiliar/ueber-uns...,Liebe Mobiliar: Parkett,,"Der neue «Liebe Mobiliar»-Spot zeigt, dass Sch...",,Die Mobiliar startet mit dem neusten «Liebe Mo...,


In [16]:
merged_text_df = merge_columns_in_dataframe(pandas_dataframe)
for i in merged_text_df["text"]:
    print (i)

Liebe Mobiliar: Parkett  Der neue «Liebe Mobiliar»-Spot zeigt, dass Schäden zu Hause immer genau dann passieren, wenn man nicht damit rechnet.  Die Mobiliar startet mit dem neusten «Liebe Mobiliar»-Spot ins neue Jahr. Die Familie im Spot hingegen startet in einen nur auf den ersten Blick gewöhnlichen Sonntagmorgen. Denn: Während die Eltern noch gemütlich vor sich hinschlummern, werden sie von Geräuschen geweckt, die definitiv nicht ins Haus gehören. Grund dafür ist ihre Tochter, die ihr Talent als Eiskunstläuferin entdeckt und mit viel Freude auslebt. Und das hinterlässt Spuren. Was immer kommt – wir helfen Ihnen rasch und unkompliziert. 


In [11]:
text = "Der neue «Liebe Mobiliar»-Spot zeigt, dass Schäden zu Hause immer genau dann passieren, wenn man nicht damit rechnet.  Die Mobiliar startet mit dem neusten «Liebe Mobiliar»-Spot ins neue Jahr. Die Familie im Spot hingegen startet in einen nur auf den ersten Blick gewöhnlichen Sonntagmorgen. Denn: Während die Eltern noch gemütlich vor sich hinschlummern, werden sie von Geräuschen geweckt, die definitiv nicht ins Haus gehören. Grund dafür ist ihre Tochter, die ihr Talent als Eiskunstläuferin entdeckt und mit viel Freude auslebt. Und das hinterlässt Spuren. Was immer kommt – wir helfen Ihnen rasch und unkompliziert."

In [ ]:
# text = "Fahren fuhren gefahren fuhr die Fahrt er fährt"

In [12]:
tokens = word_tokenize(text.lower())
tokens

['der',
 'neue',
 '«',
 'liebe',
 'mobiliar',
 '»',
 '-spot',
 'zeigt',
 ',',
 'dass',
 'schäden',
 'zu',
 'hause',
 'immer',
 'genau',
 'dann',
 'passieren',
 ',',
 'wenn',
 'man',
 'nicht',
 'damit',
 'rechnet',
 '.',
 'die',
 'mobiliar',
 'startet',
 'mit',
 'dem',
 'neusten',
 '«',
 'liebe',
 'mobiliar',
 '»',
 '-spot',
 'ins',
 'neue',
 'jahr',
 '.',
 'die',
 'familie',
 'im',
 'spot',
 'hingegen',
 'startet',
 'in',
 'einen',
 'nur',
 'auf',
 'den',
 'ersten',
 'blick',
 'gewöhnlichen',
 'sonntagmorgen',
 '.',
 'denn',
 ':',
 'während',
 'die',
 'eltern',
 'noch',
 'gemütlich',
 'vor',
 'sich',
 'hinschlummern',
 ',',
 'werden',
 'sie',
 'von',
 'geräuschen',
 'geweckt',
 ',',
 'die',
 'definitiv',
 'nicht',
 'ins',
 'haus',
 'gehören',
 '.',
 'grund',
 'dafür',
 'ist',
 'ihre',
 'tochter',
 ',',
 'die',
 'ihr',
 'talent',
 'als',
 'eiskunstläuferin',
 'entdeckt',
 'und',
 'mit',
 'viel',
 'freude',
 'auslebt',
 '.',
 'und',
 'das',
 'hinterlässt',
 'spuren',
 '.',
 'was',
 'imme

In [13]:
filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
filtered_tokens


['neue',
 '«',
 'liebe',
 'mobiliar',
 '»',
 '-spot',
 'zeigt',
 ',',
 'schäden',
 'hause',
 'immer',
 'genau',
 'passieren',
 ',',
 'rechnet',
 '.',
 'mobiliar',
 'startet',
 'neusten',
 '«',
 'liebe',
 'mobiliar',
 '»',
 '-spot',
 'neue',
 'jahr',
 '.',
 'familie',
 'spot',
 'hingegen',
 'startet',
 'ersten',
 'blick',
 'gewöhnlichen',
 'sonntagmorgen',
 '.',
 ':',
 'eltern',
 'gemütlich',
 'hinschlummern',
 ',',
 'geräuschen',
 'geweckt',
 ',',
 'definitiv',
 'haus',
 'gehören',
 '.',
 'grund',
 'dafür',
 'tochter',
 ',',
 'talent',
 'eiskunstläuferin',
 'entdeckt',
 'freude',
 'auslebt',
 '.',
 'hinterlässt',
 'spuren',
 '.',
 'immer',
 'kommt',
 '–',
 'helfen',
 'rasch',
 'unkompliziert',
 '.']

In [14]:
# from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("german") # eventuell anderen stemmer verwenden
stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
stemmed_tokens

['neu',
 '«',
 'lieb',
 'mobiliar',
 '»',
 '-spot',
 'zeigt',
 ',',
 'schad',
 'haus',
 'imm',
 'genau',
 'passi',
 ',',
 'rechnet',
 '.',
 'mobiliar',
 'startet',
 'neust',
 '«',
 'lieb',
 'mobiliar',
 '»',
 '-spot',
 'neu',
 'jahr',
 '.',
 'famili',
 'spot',
 'hingeg',
 'startet',
 'erst',
 'blick',
 'gewohn',
 'sonntagmorg',
 '.',
 ':',
 'elt',
 'gemut',
 'hinschlumm',
 ',',
 'gerausch',
 'geweckt',
 ',',
 'definitiv',
 'haus',
 'gehor',
 '.',
 'grund',
 'dafur',
 'tocht',
 ',',
 'talent',
 'eiskunstlauferin',
 'entdeckt',
 'freud',
 'auslebt',
 '.',
 'hinterlasst',
 'spur',
 '.',
 'imm',
 'kommt',
 '–',
 'helf',
 'rasch',
 'unkompliziert',
 '.']

In [16]:
from nltk.stem import WordNetLemmatizer
# from nltk.stem import SnowballStemmer
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
lemmatized_tokens

['neue',
 '«',
 'liebe',
 'mobiliar',
 '»',
 '-spot',
 'zeigt',
 ',',
 'schäden',
 'hause',
 'immer',
 'genau',
 'passieren',
 ',',
 'rechnet',
 '.',
 'mobiliar',
 'startet',
 'neusten',
 '«',
 'liebe',
 'mobiliar',
 '»',
 '-spot',
 'neue',
 'jahr',
 '.',
 'familie',
 'spot',
 'hingegen',
 'startet',
 'ersten',
 'blick',
 'gewöhnlichen',
 'sonntagmorgen',
 '.',
 ':',
 'eltern',
 'gemütlich',
 'hinschlummern',
 ',',
 'geräuschen',
 'geweckt',
 ',',
 'definitiv',
 'haus',
 'gehören',
 '.',
 'grund',
 'dafür',
 'tochter',
 ',',
 'talent',
 'eiskunstläuferin',
 'entdeckt',
 'freude',
 'auslebt',
 '.',
 'hinterlässt',
 'spuren',
 '.',
 'immer',
 'kommt',
 '–',
 'helfen',
 'rasch',
 'unkompliziert',
 '.']

In [20]:
import spacy
nlp = spacy.load("de_core_news_sm")

# Beispieltext
# text = "fahren fuhren gefahren fuhr die Fahrt"

# Verarbeiten des Textes mit SpaCy
doc = nlp(text)

filtered_tokens = [
        token for token in doc if token not in stopwords.words("german")
    ]
# Lemmatisieren der Tokens
lemmatized_tokens = [token.lemma_ for token in filtered_tokens]
lemmatized_tokens

['der',
 'neu',
 '--',
 'Liebe',
 'Mobiliar»-Spot',
 'zeigen',
 '--',
 'daß',
 'Schaden',
 'zu',
 'Haus',
 'immer',
 'genau',
 'dann',
 'passieren',
 '--',
 'wenn',
 'man',
 'nicht',
 'damit',
 'rechnen',
 '--',
 ' ',
 'der',
 'Mobiliar',
 'starten',
 'mit',
 'der',
 'neust',
 '--',
 'Liebe',
 'Mobiliar»-Spot',
 'in',
 'neu',
 'Jahr',
 '--',
 'der',
 'Familie',
 'in',
 'Spot',
 'hingegen',
 'starten',
 'in',
 'ein',
 'nur',
 'auf',
 'der',
 'erster',
 'Blick',
 'gewöhnlich',
 'Sonntagmorg',
 '--',
 'denn',
 '--',
 'während',
 'der',
 'Eltern',
 'noch',
 'gemütlich',
 'vor',
 'sich',
 'hinschlummern',
 '--',
 'werden',
 'sie',
 'von',
 'Geräusche',
 'wecken',
 '--',
 'der',
 'definitiv',
 'nicht',
 'in',
 'Haus',
 'gehören',
 '--',
 'Grund',
 'dafür',
 'sein',
 'ihr',
 'Tochter',
 '--',
 'der',
 'ihr',
 'Talent',
 'als',
 'Eiskunstläuferin',
 'entdecken',
 'und',
 'mit',
 'viel',
 'Freude',
 'ausleben',
 '--',
 'und',
 'der',
 'hinterlässen',
 'Spur',
 '--',
 'was',
 'immer',
 'kommen',

In [15]:
# stemmed_tokens
processed_text = " ".join(stemmed_tokens)
processed_text

'neu « lieb mobiliar » -spot zeigt , schad haus imm genau passi , rechnet . mobiliar startet neust « lieb mobiliar » -spot neu jahr . famili spot hingeg startet erst blick gewohn sonntagmorg . : elt gemut hinschlumm , gerausch geweckt , definitiv haus gehor . grund dafur tocht , talent eiskunstlauferin entdeckt freud auslebt . hinterlasst spur . imm kommt – helf rasch unkompliziert .'

In [31]:

scores = analyzer.polarity_scores(processed_text)
print(scores)
pd.Series(scores)

{'neg': 0.0, 'neu': 0.946, 'pos': 0.054, 'compound': 0.4215}


neg         0.0000
neu         0.9460
pos         0.0540
compound    0.4215
dtype: float64

In [114]:
from transformers import AutoTokenizer
import torch.nn.functional as F

model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
classifier = pipeline(
    model=model_name,
    # return_all_scores=True, ist deprecated
    tokenizer = AutoTokenizer.from_pretrained(model_name),
    top_k=None # gibt alle scores zurück
)

X_train = [text, "Ich bin Roman und 28 Jahre alt.", "Ich liebe dich!"]
res = classifier(X_train)

print(res)
tokenizer = AutoTokenizer.from_pretrained(model_name)
batch = tokenizer(X_train, padding=True, truncation=True, max_length=800, return_tensors="pt")

with torch.no_grad():
    outputs = model_name(**batch),
    print(outputs)
    predictions = F.softmax(output.logits, dim=1),
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)


# list_list_of_dicts_of_tuples = res
# for list_of_dicts_of_tuples in list_list_of_dicts_of_tuples:
#     for dicts in list_of_dicts_of_tuples:
#         mydict[dicts["label"]] = dicts["score"]
# print(pd.Series(mydict))

[[{'label': 'negative', 'score': 0.6421080827713013}, {'label': 'positive', 'score': 0.27281466126441956}, {'label': 'neutral', 'score': 0.0850772112607956}], [{'label': 'positive', 'score': 0.49148350954055786}, {'label': 'neutral', 'score': 0.25997069478034973}, {'label': 'negative', 'score': 0.2485458105802536}], [{'label': 'positive', 'score': 0.9740146994590759}, {'label': 'neutral', 'score': 0.015805009752511978}, {'label': 'negative', 'score': 0.01018031220883131}]]


TypeError: 'str' object is not callable

In [106]:
for list_of_dicts_of_tuples in classifier(text):
    for dicts in list_of_dicts_of_tuples:
        mydict[dicts["label"]] = dicts["score"]
print(pd.Series(mydict))
print()
for list_of_dicts_of_tuples in classifier("Ich liebe dich!"):
    for dicts in list_of_dicts_of_tuples:
        mydict[dicts["label"]] = dicts["score"]
print(pd.Series(mydict))
print()
for list_of_dicts_of_tuples in classifier("Ich bin Roman und 28 Jahre alt."):
    for dicts in list_of_dicts_of_tuples:
        mydict[dicts["label"]] = dicts["score"]
print(pd.Series(mydict))

negative    0.642108
positive    0.272815
neutral     0.085077
dtype: float64

negative    0.010180
positive    0.974015
neutral     0.015805
dtype: float64

negative    0.248546
positive    0.491484
neutral     0.259971
dtype: float64


In [107]:
classifier = pipeline(
    "sentiment-analysis",
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    # return_all_scores=True, ist deprecated
    top_k=None, # gibt alle scores zurück
    # tokenizer=tok
)

for list_of_dicts_of_tuples in classifier(text):
    for dicts in list_of_dicts_of_tuples:
        mydict[dicts["label"]] = dicts["score"]
print(pd.Series(mydict))
print()
for list_of_dicts_of_tuples in classifier("Ich liebe dich!"):
    for dicts in list_of_dicts_of_tuples:
        mydict[dicts["label"]] = dicts["score"]
print(pd.Series(mydict))
print()
for list_of_dicts_of_tuples in classifier("Ich bin Roman und 28 Jahre alt."):
    for dicts in list_of_dicts_of_tuples:
        mydict[dicts["label"]] = dicts["score"]
print(pd.Series(mydict))

negative    0.642108
positive    0.272815
neutral     0.085077
dtype: float64

negative    0.010180
positive    0.974015
neutral     0.015805
dtype: float64

negative    0.248546
positive    0.491484
neutral     0.259971
dtype: float64


In [88]:
from transformers import pipeline
import torch

distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    top_k=None
)

# distilled_student_sentiment_classifier(text)
distilled_student_sentiment_classifier(text)

[[{'label': 'negative', 'score': 0.6421080827713013},
  {'label': 'positive', 'score': 0.27281466126441956},
  {'label': 'neutral', 'score': 0.0850772112607956}]]

In [121]:

def get_sentiment_hf(text_string):
    classifier = pipeline(
        model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
        # return_all_scores=True, ist deprecated
        truncation=True,
        top_k=None # gibt alle scores zurück
    )
    list_list_of_dicts_of_tuples = classifier(text_string)
    mydict = {}
    for list_of_dicts_of_tuples in list_list_of_dicts_of_tuples:
        for dicts in list_of_dicts_of_tuples:
            mydict[dicts["label"]] = dicts["score"]
    return pd.Series(mydict)


def add_sentiment_cols_hf(dataframe, text_col="text"):
    sentiment_cols = dataframe[text_col].apply(get_sentiment)
    dataframe = pd.concat([dataframe, sentiment_cols], axis=1)
    return dataframe

In [122]:
def get_analysis_data_hf(json_file):
    pandas_df = pd.read_json(json_file)
    filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'page_title'])] != "").any(axis=1)] # exkludiert alle spalten von der Analyse die "" in den spalten ausser "page_title" haben 
    data = merge_columns_in_dataframe(filtered_df)
    return add_sentiment_cols(data)

In [123]:
get_analysis_data_hf("hans.json")

,url,page_title,text,negative,positive,neutral
1,https://www.mobiliar.ch/versicherungen-und-vor...,Selbstunfall mit dem Auto – was nun?,Selbstunfall mit dem Auto – was nun? Selbst s...,0.785902,0.107085,0.107012
2,https://www.mobiliar.ch/versicherungen-und-vor...,Invalidität oder Todesfall: So sorgen Sie vor,Invalidität oder Todesfall: So sorgen Sie vor ...,0.488104,0.375045,0.136852
3,https://www.mobiliar.ch/versicherungen-und-vor...,Schäden an Ihrer Mietwohnung,Schäden an Ihrer Mietwohnung Für welche Schäde...,0.795895,0.108779,0.095326


In [129]:
get_analysis_data("hans.json")

,url,page_title,text,neg,neu,pos,compound,judgement
1,https://www.mobiliar.ch/versicherungen-und-vor...,Selbstunfall mit dem Auto – was nun?,selbstunfall auto – ? schuld ! sekund aufgepas...,0.025,0.975,0.000,-0.5627,negative
2,https://www.mobiliar.ch/versicherungen-und-vor...,Invalidität oder Todesfall: So sorgen Sie vor,"invaliditat todesfall : sorg optimal vorsorg ,...",0.000,0.963,0.037,0.7820,positive
3,https://www.mobiliar.ch/versicherungen-und-vor...,Schäden an Ihrer Mietwohnung,schad mietwohn schad haft mieterin miet ? freu...,0.039,0.961,0.000,-0.9749,negative
